In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [16]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import ast


In [17]:
movies = pd.read_csv("/kaggle/input/movie-recommand/tmdb_5000_movies.csv")
credits = pd.read_csv("/kaggle/input/movie-recommand/tmdb_5000_credits.csv")

print("Movies shape:", movies.shape)
print("Credits shape:", credits.shape)


Movies shape: (4803, 20)
Credits shape: (4803, 4)


In [18]:

df = movies.merge(credits, left_on="title", right_on="title")
df = df[['movie_id','title','overview','genres','keywords','cast','crew']]
df.head()


,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [19]:
def convert(obj):
    try:
        L = []
        for i in ast.literal_eval(obj):
            L.append(i['name'])
        return L
    except:
        return []

df['genres'] = df['genres'].apply(convert)
df['keywords'] = df['keywords'].apply(convert)


def fetch_cast(obj):
    try:
        L = []
        for i in ast.literal_eval(obj)[:3]:
            L.append(i['name'])
        return L
    except:
        return []

df['cast'] = df['cast'].apply(fetch_cast)


def fetch_director(obj):
    try:
        L = []
        for i in ast.literal_eval(obj):
            if i['job'] == 'Director':
                return [i['name']]
        return []
    except:
        return []

df['crew'] = df['crew'].apply(fetch_director)


In [20]:
df['overview'] = df['overview'].fillna('')
df['tags'] = df['overview'] + " " + df['genres'].apply(lambda x: " ".join(x)) + " " + \
             df['keywords'].apply(lambda x: " ".join(x)) + " " + \
             df['cast'].apply(lambda x: " ".join(x)) + " " + \
             df['crew'].apply(lambda x: " ".join(x))

df = df[['movie_id','title','tags']]
df.head()


,movie_id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."


In [21]:
tfidf = TfidfVectorizer(stop_words="english", max_features=5000)
tfidf_matrix = tfidf.fit_transform(df['tags'])


In [22]:
title_to_index = pd.Series(df.index, index=df['title']).to_dict()

def recommend(movie_title, top_n=5):
    if movie_title not in title_to_index:
        return ["Movie not found!"]
    
    idx = title_to_index[movie_title]
    sim_scores = linear_kernel(tfidf_matrix[idx], tfidf_matrix).flatten()
    sim_indices = sim_scores.argsort()[::-1][1:top_n+1]
    
    recs = []
    for i in sim_indices:
        recs.append((df.iloc[i]['title'], round(sim_scores[i], 3)))
    return recs


In [25]:

title_to_index = pd.Series(df.index, index=df['title']).to_dict()

def recommend(movie_title, top_n=5):
    if movie_title not in title_to_index:
        return ["Movie not found!"]
    
    idx = title_to_index[movie_title]
    sim_scores = linear_kernel(tfidf_matrix[idx], tfidf_matrix).flatten()
    sim_indices = sim_scores.argsort()[::-1][1:top_n+1]
    
    recs = []
    for i in sim_indices:
        reasons = []
        overlap = set(df.loc[idx, 'tags'].split()) & set(df.loc[i, 'tags'].split())
        if overlap:
            reasons.append(f"Common words: {list(overlap)[:3]}")  # show up to 3 words
        recs.append({
            'title': df.iloc[i]['title'],
            'similarity': round(sim_scores[i], 3),
            'reasons': reasons if reasons else ["General similarity"]
        })
    return recs


In [26]:
print("Recommendations for Avatar:")
print(recommend("Avatar"))

print("\nRecommendations for Inception:")
print(recommend("Inception"))

print("\nRecommendations for Titanic:")
print(recommend("Titanic"))


Recommendations for Avatar:
[{'title': 'Aliens', 'similarity': 0.424, 'reasons': ["Common words: ['and', 'James', 'Science']"]}, {'title': 'Alien', 'similarity': 0.363, 'reasons': ["Common words: ['a', 'Fiction', 'Action']"]}, {'title': 'Moonraker', 'similarity': 0.349, 'reasons': ["Common words: ['a', 'Fiction', 'Adventure']"]}, {'title': 'Alien³', 'similarity': 0.346, 'reasons': ["Common words: ['a', 'Fiction', 'Action']"]}, {'title': 'Silent Running', 'similarity': 0.311, 'reasons': ["Common words: ['future', 'a', 'Adventure']"]}]

Recommendations for Inception:
[{'title': 'Hesher', 'similarity': 0.172, 'reasons': ["Common words: ['who', 'of', 'the']"]}, {'title': 'Don Jon', 'similarity': 0.164, 'reasons': ["Common words: ['his', 'Joseph', 'to']"]}, {'title': 'Cypher', 'similarity': 0.145, 'reasons': ["Common words: ['a', 'Fiction', 'Mystery']"]}, {'title': '(500) Days of Summer', 'similarity': 0.144, 'reasons': ["Common words: ['his', 'Joseph', 'to']"]}, {'title': 'The Helix... Loa